**Прогноз цен на авиабилеты**

Датасет:
https://www.kaggle.com/competitions/aviachipta-narxini-bashorat-qilish/data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data = pd.read_csv("train_data.csv", index_col='id')
test_data = pd.read_csv("test_data.csv", index_col='id')
submission = pd.read_csv("sample_solution.csv")
df = train_data

df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
id,,,,,,,,,,,
1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 1 to 20000
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   airline           20000 non-null  object 
 1   flight            20000 non-null  object 
 2   source_city       20000 non-null  object 
 3   departure_time    20000 non-null  object 
 4   stops             20000 non-null  object 
 5   arrival_time      20000 non-null  object 
 6   destination_city  20000 non-null  object 
 7   class             20000 non-null  object 
 8   duration          20000 non-null  float64
 9   days_left         20000 non-null  int64  
 10  price             20000 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 1.8+ MB


In [ ]:
train_data = train_data.drop('airline', axis=1)
train_data = train_data.drop('flight', axis=1)
train_data = train_data.drop('duration', axis=1)
train_data = train_data.drop('arrival_time', axis=1)

In [ ]:
train_data

,source_city,departure_time,stops,destination_city,class,days_left,price
id,,,,,,,
1,Bangalore,Early_Morning,one,Mumbai,Economy,21,7212
2,Hyderabad,Evening,zero,Kolkata,Economy,7,5292
3,Bangalore,Morning,one,Delhi,Business,5,60553
4,Kolkata,Morning,one,Hyderabad,Economy,28,5760
5,Chennai,Early_Morning,zero,Mumbai,Economy,4,10712
...,...,...,...,...,...,...,...
19996,Bangalore,Night,one,Mumbai,Economy,45,3153
19997,Kolkata,Morning,one,Delhi,Economy,24,3911
19998,Chennai,Early_Morning,two_or_more,Bangalore,Economy,17,14822


In [ ]:
le = LabelEncoder()
train_data['source_city'] = le.fit_transform(train_data['source_city'])
train_data['destination_city'] = le.fit_transform(train_data['destination_city'])
train_data['class'] = le.fit_transform(train_data['class'])
train_data['stops'] = le.fit_transform(train_data['stops'])
train_data['class'] = le.fit_transform(train_data['class'])
train_data['departure_time'] = le.fit_transform(train_data['departure_time'])

In [ ]:
train_data.head()

,source_city,departure_time,stops,destination_city,class,days_left,price
id,,,,,,,
1,0,1,0,5,1,21,7212
2,3,2,2,4,1,7,5292
3,0,4,0,2,0,5,60553
4,4,4,0,3,1,28,5760
5,1,1,2,5,1,4,10712


In [ ]:
test_data = test_data.drop('airline', axis=1)
test_data = test_data.drop('flight', axis=1)
test_data = test_data.drop('duration', axis=1)
test_data = test_data.drop('arrival_time', axis=1)

In [ ]:
test_data['source_city'] = le.fit_transform(test_data['source_city'])
test_data['destination_city'] = le.fit_transform(test_data['destination_city'])
test_data['class'] = le.fit_transform(test_data['class'])
test_data['stops'] = le.fit_transform(test_data['stops'])
test_data['class'] = le.fit_transform(test_data['class'])
test_data['departure_time'] = le.fit_transform(test_data['departure_time'])

In [ ]:
test_data.head()

,source_city,departure_time,stops,destination_city,class,days_left
id,,,,,,
1,4,2,0,2,0,2
2,2,1,0,5,0,34
3,5,1,2,1,0,30
4,3,5,0,2,1,26
5,4,5,0,5,1,35


In [ ]:
X_train = train_data.drop('price', axis=1)
y_train = train_data['price']

In [ ]:
train_set, test_set = train_test_split(train_data, test_size=0.2, random_state=42)

In [ ]:
num_attribs=['stops', 'class', 'days_left', 'source_city', 'departure_time', 'destination_city']
cat_attribs=[]

num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs)
])

In [ ]:
X_prep = full_pipeline.fit_transform(X_train)

In [ ]:
test_prep = full_pipeline.transform(test_data)

In [ ]:
LR_model = LinearRegression()
LR_model.fit(X_prep, y_train)
lr_predict=LR_model.predict(X_prep)
mse=mean_squared_error(lr_predict,y_train)
error=np.sqrt(mse)
print(f"MSE: {error.round()}")

y_test_predicted_LR = LR_model.predict(test_prep)
y_test_predicted_LR

MSE: 7234.0


array([56776.37461541, 52223.02487141, 45086.58252703, ...,
       52137.45548065, 51806.27942576, 50281.66009893])

In [ ]:
RF_model = RandomForestRegressor()
RF_model.fit(X_prep, y_train)
lr_predict=RF_model.predict(X_prep)
mse=mean_squared_error(lr_predict,y_train)
error=np.sqrt(mse)
print(f"MSE: {error.round()}")

y_test_predicted_RFR = RF_model.predict(test_prep)
y_test_predicted_RFR

MSE: 3396.0


array([61529.13333333, 51775.84083333, 23306.56      , ...,
       61139.883     , 47337.28733333, 58498.55933333])

In [ ]:
submission['price'] = y_test_predicted_RFR
submission

,id,price
0,1,61529.133333
1,2,51775.840833
2,3,23306.560000
3,4,5851.891429
4,5,5930.850317
...,...,...
4995,4996,62200.005000
4996,4997,3903.280000
4997,4998,61139.883000
4998,4999,47337.287333


In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
import joblib

joblib.dump(full_pipeline, "full_pipeline.pkl")

['full_pipeline.pkl']

In [ ]:
joblib.dump(RF_model, "RF_model.pkl")

['RF_model.pkl']

**ТЕСТ**

Пример того, как модель принимает на вход переменные

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
import pickle
import joblib

In [ ]:
loaded_model = joblib.load("RF_model.pkl")

In [ ]:
full_pipeline = joblib.load("full_pipeline.pkl")

In [ ]:
id_fligth = 1
airline = ""
flight = ""
source_city = "Delhi"
departure_time = "Early_Morning"
stops = "one"
arrival_time = "Morning"
destination_city = "Mumbai"
class_flight = "Business"
duration = 0
days_left = 3

df1 = pd.DataFrame([[airline, flight, source_city, departure_time, stops, arrival_time, destination_city, class_flight, duration, days_left]],
                   columns=["airline", "flight", "source_city", "departure_time", "stops", "arrival_time", "destination_city", "class", "duration", "days_left"])
df1.index.name = 'id'

In [ ]:
le = LabelEncoder()
df1['airline'] = le.fit_transform(df1['airline'])
df1['source_city'] = le.fit_transform(df1['source_city'])
df1['destination_city'] = le.fit_transform(df1['destination_city'])
df1['class'] = le.fit_transform(df1['class'])
df1['stops'] = le.fit_transform(df1['stops'])
df1['class'] = le.fit_transform(df1['class'])
df1['departure_time'] = le.fit_transform(df1['departure_time'])
df1['arrival_time'] = le.fit_transform(df1['arrival_time'])

In [ ]:
X_test_prep1 = full_pipeline.transform(df1)
y_predicted = loaded_model.predict(X_test_prep1)

y_predicted

array([62208.35333333])